In [5]:
import requests
import pandas as pd
import pydantic
import datetime
from pathlib import Path

from secret import api_key

In [15]:
class ElexonRequest(pydantic.BaseModel):
    report: str
    date: datetime.date
    api_key: pydantic.SecretStr = pydantic.SecretStr(api_key)
    service_type: str = 'csv'
    period: str = '*'

req = ElexonRequest(report='B1780', date='2020-01-01')
url = f"https://api.bmreports.com/BMRS/{req.report}/v1?APIKey={req.api_key.get_secret_value()}&Period={req.period}&SettlementDate={req.date.isoformat()}&ServiceType={req.service_type}"
res = requests.get(url)
assert res.status_code == 200
# print(res.text[:512])
# print(res.text)

In [16]:
# set the path desired to save the file
fi = Path().cwd()/'data'/f'{req.report}_{req.date.isoformat()}.csv'
# make the parent directory if it doesn't exist
fi.parent.mkdir(parents=True, exist_ok=True)
# write the reponse to the file 
fi.write_text(res.text)


9347

In [20]:
data = pd.read_csv('data/B1770_2020-01-01.csv', skiprows=4)
# drop the last row as it has nan values
data = data.dropna(axis=0)
data.tail()



,*DocumentID,DocumentRevNum,ActiveFlag,ProcessType,DocumentType,Resolution,CurveType,PriceCategory,ImbalancePriceAmount,SettlementPeriod,SettlementDate,ControlArea,BusinessType,TimeSeriesID,DocumentStatus
91,ELX-EMFIP-IMBP-22438191,1.0,Y,Realised,Imbalance prices,PT30M,Sequential fixed size block,Insufficient balance,29.37006,3.0,2020-01-01,10YGB----------A,Balance energy deviation,ELX-EMFIP-IMBP-TS-1,Final
92,ELX-EMFIP-IMBP-22438072,1.0,Y,Realised,Imbalance prices,PT30M,Sequential fixed size block,Excess balance,51.00000,2.0,2020-01-01,10YGB----------A,Balance energy deviation,ELX-EMFIP-IMBP-TS-2,Final
93,ELX-EMFIP-IMBP-22438072,1.0,Y,Realised,Imbalance prices,PT30M,Sequential fixed size block,Insufficient balance,51.00000,2.0,2020-01-01,10YGB----------A,Balance energy deviation,ELX-EMFIP-IMBP-TS-1,Final
94,ELX-EMFIP-IMBP-22437930,1.0,Y,Realised,Imbalance prices,PT30M,Sequential fixed size block,Excess balance,50.90000,1.0,2020-01-01,10YGB----------A,Balance energy deviation,ELX-EMFIP-IMBP-TS-2,Final
95,ELX-EMFIP-IMBP-22437930,1.0,Y,Realised,Imbalance prices,PT30M,Sequential fixed size block,Insufficient balance,50.90000,1.0,2020-01-01,10YGB----------A,Balance energy deviation,ELX-EMFIP-IMBP-TS-1,Final


In [21]:
# function to send request to Elexon, clean save the file

def send_elexon_request(req: ElexonRequest) -> pd.DataFrame:
    # define the url
    url = f"https://api.bmreports.com/BMRS/{req.report}/v1?APIKey={req.api_key.get_secret_value()}&Period={req.period}&SettlementDate={req.date.isoformat()}&ServiceType={req.service_type}"
    # make the request
    res = requests.get(url)
    # check the status code
    assert res.status_code == 200
    
    # set the path desired to save the file
    fi = Path().cwd()/'data'/f'{req.report}_{req.date.isoformat()}.csv'
    # make the parent directory if it doesn't exist
    fi.parent.mkdir(parents=True, exist_ok=True)
    # write the reponse to the file 
    fi.write_text(res.text)

    # open the file and clean
    data = pd.read_csv(fi, skiprows=4)
    # change the column names so that they are consistent
    # some have a space in the name and some don't
    data.columns = [c.replace(' ', '') for c in data.columns]
    # drop rows if they have nan values and a valid settlement period
    data = data.dropna(axis=0, subset=['SettlementDate'])

    return data

    



In [22]:
report = 'B1770'
date = '2020-01-01'

req = ElexonRequest(report=report, date=date)
data = send_elexon_request(req)

data.head()

,*DocumentID,DocumentRevNum,ActiveFlag,ProcessType,DocumentType,Resolution,CurveType,PriceCategory,ImbalancePriceAmount,SettlementPeriod,SettlementDate,ControlArea,BusinessType,TimeSeriesID,DocumentStatus
0,ELX-EMFIP-IMBP-22444358,1.0,Y,Realised,Imbalance prices,PT30M,Sequential fixed size block,Excess balance,49.95,48.0,2020-01-01,10YGB----------A,Balance energy deviation,ELX-EMFIP-IMBP-TS-2,Final
1,ELX-EMFIP-IMBP-22444358,1.0,Y,Realised,Imbalance prices,PT30M,Sequential fixed size block,Insufficient balance,49.95,48.0,2020-01-01,10YGB----------A,Balance energy deviation,ELX-EMFIP-IMBP-TS-1,Final
2,ELX-EMFIP-IMBP-22444205,1.0,Y,Realised,Imbalance prices,PT30M,Sequential fixed size block,Excess balance,50.05,47.0,2020-01-01,10YGB----------A,Balance energy deviation,ELX-EMFIP-IMBP-TS-2,Final
3,ELX-EMFIP-IMBP-22444205,1.0,Y,Realised,Imbalance prices,PT30M,Sequential fixed size block,Insufficient balance,50.05,47.0,2020-01-01,10YGB----------A,Balance energy deviation,ELX-EMFIP-IMBP-TS-1,Final
4,ELX-EMFIP-IMBP-22444056,1.0,Y,Realised,Imbalance prices,PT30M,Sequential fixed size block,Excess balance,49.45,46.0,2020-01-01,10YGB----------A,Balance energy deviation,ELX-EMFIP-IMBP-TS-2,Final
